In [1]:
import numpy as np
import torch
from env import MultiAgentEnv

In [3]:
X_train, y_train = np.load('data/X_train.npy', allow_pickle=True), np.load('data/y_train.npy', allow_pickle=True)
env = MultiAgentEnv(X_train, y_train)

In [4]:
from tqdm import tqdm

In [ ]:
def collect_samples(env, horizon, disable_tqdm=False, print_done_states=False):
    s, _ = env.reset()
    #dataset = []
    S = []
    A = []
    R = []
    S2 = []
    D = []

    for _ in tqdm(range(horizon), disable=disable_tqdm):
        a = env.sample_action()
        s2, r, done, _ = env.step(a)
        #dataset.append((s,a,r,s2,done,trunc))
        # state = current_position + embedding
        s = torch.concat(env.X, s, axis = 0)
        S.append(s)
        A.append(a)
        R.append(r)
        S2.append(s2)
        D.append(done)
        
        for i in range(env.num_agents):
            if done[i]:
                env.state[i] = 0
            else:
                env.state[i] = s2[i]

    S = np.array(S)
    A = np.array(A).reshape((-1,1))
    R = np.array(R)
    S2= np.array(S2)
    D = np.array(D)
    return S, A, R, S2, D

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

def rf_fqi(S, A, R, S2, D, iterations, nb_actions, gamma, disable_tqdm=False):
    nb_samples = S.shape[0]
    Qfunctions = []
    SA = np.append(S,A,axis=1)
    for iter in tqdm(range(iterations), disable=disable_tqdm):
        if iter==0:
            value=R.copy()
        else:
            Q2 = np.zeros((nb_samples,nb_actions))
            for a2 in range(nb_actions):
                A2 = a2*np.ones((S.shape[0],1))
                S2A2 = np.append(S2,A2,axis=1)
                Q2[:,a2] = Qfunctions[-1].predict(S2A2)
            max_Q2 = np.max(Q2,axis=1)
            value = R + gamma*(1-D)*max_Q2
        Q = RandomForestRegressor()
        Q.fit(SA,value)
        Qfunctions.append(Q)
    return Qfunctions
